# LangChin的链式调用

In [1]:
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser # 导入标准输出组件

model = init_chat_model(
    model="Qwen/Qwen3-8B",
    model_provider="openai",
    base_url="https://api.siliconflow.cn/v1/",
    api_key="sk-vjklrvkomqgjluwtfxssjnmetzvtwgfqgdgcyiycymhpjpzd", #你注册的硅基流动api_key
)

# 搭建链条，把model和字符串输出解析器组件连接在一起
basic_qa_chain = model | StrOutputParser()

# 查看输出结果
question = "你好，请你介绍一下你自己。"
result = basic_qa_chain.invoke(question)

print(result)

c:\ProgramData\miniconda3\envs\clip\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\ProgramData\miniconda3\envs\clip\lib\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]




你好！我是通义千问，由通义实验室研发的超大规模语言模型。我的知识来源于阿里巴巴集团在互联网上收集的海量文本数据，经过深度学习和训练，我能够理解和生成自然语言，具备多语言支持能力，包括中文、英文、日文、韩文等。

我可以帮助你完成各种任务，比如回答问题、创作文字、编程、逻辑推理等。无论你是需要学习知识、工作协助，还是日常交流，我都可以提供帮助。如果你有任何问题或需要支持，请随时告诉我！


In [2]:
from langchain.chat_models import init_chat_model
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = init_chat_model(
    model="Qwen/Qwen3-8B",
    model_provider="openai",
    base_url="https://api.siliconflow.cn/v1/",
    api_key="sk-vjklrvkomqgjluwtfxssjnmetzvtwgfqgdgcyiycymhpjpzd", #你注册的硅基流动api_key
)

prompt_template = ChatPromptTemplate([
    ("system", "你是一个乐意助人的助手，请根据用户的问题给出回答"),
    ("user", "这是用户的问题： {topic}， 请用 yes 或 no 来回答")
])

# 直接使用模型 + 输出解析器
bool_qa_chain = prompt_template | model | StrOutputParser()
# 测试
question = "请问 1 + 1 是否 大于 2？"
result = bool_qa_chain.invoke({'topic':question})
print(result)



no


In [4]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

schemas = [ # 构建结构化数据模板
    ResponseSchema(name="name", description="用户的姓名"),
    ResponseSchema(name="age", description="用户的年龄")
]

parser = StructuredOutputParser.from_response_schemas(schemas) # 根据模板生成解析器

model = init_chat_model(
    model="Qwen/Qwen3-8B",
    model_provider="openai",
    base_url="https://api.siliconflow.cn/v1/",
    api_key="sk-vjklrvkomqgjluwtfxssjnmetzvtwgfqgdgcyiycymhpjpzd", #你注册的硅基流动api_key
)

prompt = PromptTemplate.from_template(
    "请根据以下内容提取用户信息，并返回 JSON 格式：\n{input}\n\n{format_instructions}"
) # 这是另一种使用占位符的提示词模板表示方式

chain = (
    prompt.partial(format_instructions=parser.get_format_instructions()) 
    | model
    | parser
)

result = chain.invoke({"input": "用户叫李雷，今年25岁，是一名工程师。"}) # 输入input, format_instructions前面已经赋值
print(result)

{'name': '李雷', 'age': '25'}


In [11]:
parser = StructuredOutputParser.from_response_schemas(schemas) 
a = prompt.partial(format_instructions=parser.get_format_instructions())

In [10]:
print(parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"name": string  // 用户的姓名
	"age": string  // 用户的年龄
}
```


In [13]:
a

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"name": string  // 用户的姓名\n\t"age": string  // 用户的年龄\n}\n```'}, template='请根据以下内容提取用户信息，并返回 JSON 格式：\n{input}\n\n{format_instructions}')

In [1]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

model = init_chat_model(
    model="Qwen/Qwen3-8B",
    model_provider="openai",
    base_url="https://api.siliconflow.cn/v1/",
    api_key="sk-vjklrvkomqgjluwtfxssjnmetzvtwgfqgdgcyiycymhpjpzd", #你注册的硅基流动api_key
)

################################## 生成新闻的Chain ##################################
# 新闻生成的提示词模版
news_prompt = PromptTemplate.from_template(
    "请根据以下新闻标题撰写一段简短的新闻内容（100字以内）：\n\n 标题：{title}"
)

# 生成新闻的chain
chain01 = news_prompt | model
################################## 生成新闻的Chain ----End ##################################

################################## 新闻内容提取的Chain ##################################
# 新闻内容提取的结构化数据模版
schema = [
    ResponseSchema(name="time", description="事件发生的时间"),
    ResponseSchema(name="location", description="事件发生的地点"),
    ResponseSchema(name="event", description="发生的具体事件"),
]

# 根据定义的的结构化模版生成解释器
parser = StructuredOutputParser.from_response_schemas(schema)

# 定义prompt
summary_prompt = PromptTemplate.from_template(
    "请从下面这段新闻内容中提取关键信息，并返回结构化JSON格式：\n\n{news}\n\n{format_instructions}"
)
# 新闻内容提取的chain
chain02 = (
    summary_prompt.partial(format_instructions = parser.get_format_instructions()) | model | parser
)
################################## 新闻内容提取的Chain ----End ##################################
# 组合成一个复合 Chain
full_chain = chain01 | chain02

# 调用复合链
result = full_chain.invoke({"title": "苹果公司在加州发布新款AI芯片"})


c:\ProgramData\miniconda3\envs\clip\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\ProgramData\miniconda3\envs\clip\lib\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [20]:
result

{'time': '近日',
 'location': '加州',
 'event': '苹果公司发布新款AI芯片，专为提升设备智能性能设计，强化机器学习运算能力，并计划搭载于iPhone、Mac及iPad产品线'}

### 自定义组件

In [2]:
from langchain_core.runnables import RunnableLambda

def debug_print(x):
    print('中间结果（新闻正文）：', x)
    return x

debug_node = RunnableLambda(debug_print)

# 组合成一个复合 Chain
full_chain = chain01 | debug_node | chain02

# 调用复合链
result = full_chain.invoke({"title": "苹果公司在加州发布新款AI芯片"})
print(result)

中间结果（新闻正文）： content='\n\n苹果公司于周二在加州举行发布会，正式推出自研AI芯片A17，采用3nm制程工艺，专为提升设备AI算力与能效比设计。该芯片将搭载于新一代iPhone 15系列及Mac产品线，标志着苹果在移动端AI技术布局的深化，进一步强化其生态系统的智能化体验。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 442, 'prompt_tokens': 39, 'total_tokens': 481, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 368, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '019a04895042ded1bc91afea1a709bd0', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--5f98ede9-c6d1-4d87-953a-3e33d3ba26dd-0' usage_metadata={'input_tokens': 39, 'output_tokens': 442, 'total_tokens': 481, 'input_token_details': {}, 'output_token_details': {'reasoning': 368}}
{'time': '周二', 'location': '加州', 'event': '苹果公司正式推出自研AI芯片A17'}
